## Imports

In [1]:
import os
from dotenv import load_dotenv

import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



## Config

In [2]:
# Fix random state for reproducibility
RANDOM_STATE = 42

# Load environment variables
load_dotenv()

# Get project root (one level up from notebooks/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) if 'notebooks' in os.getcwd() else os.getcwd()

# Get preprocessed directory from env or use default
preprocessed_dir = os.getenv('PREPROCESSED_DATA_DIR', 'data/preprocessed')

# If path is relative, make it relative to project root
if not os.path.isabs(preprocessed_dir):
    preprocessed_dir = os.path.join(project_root, preprocessed_dir)

# Check if preprocessed data exists
print(f'📁 Project root: {project_root}')
print(f'📁 Preprocessed dir: {preprocessed_dir}')

if not os.path.exists(preprocessed_dir):
    print(f'\n⚠️  Warning: Preprocessed directory not found!')
    print('   Run from project root: python3 -m src.main')
else:
    print(f'\n✅ Preprocessed directory found')


📁 Project root: /home/mle-user/mle_projects/mle-project-sprint-4
📁 Preprocessed dir: /home/mle-user/mle_projects/mle-project-sprint-4/data/preprocessed

✅ Preprocessed directory found


## Preprocessed Data Loading

In [3]:
# ---------- Load datasets ---------- #

# Check if preprocessed files exist
items_path = os.path.join(preprocessed_dir, 'items.parquet')
events_path = os.path.join(preprocessed_dir, 'events.parquet')
tracks_catalog_path = os.path.join(preprocessed_dir, 'tracks_catalog_clean.parquet')

if not os.path.exists(items_path) or not os.path.exists(events_path):
    print('❌ Preprocessed data not found!')
    print(f'   Looking for files in: {preprocessed_dir}')
    print('\n📝 To generate preprocessed data, run from terminal:')
    print('   cd /home/mle-user/mle_projects/mle-project-sprint-4')
    print('   python3 -m src.main')
    print('\n   This will:')
    print('   1. Download raw data (if needed)')
    print('   2. Preprocess and clean the data')
    print('   3. Generate items.parquet and events.parquet')
    raise FileNotFoundError(f'Preprocessed data not found in {preprocessed_dir}')

# Load preprocessed data
print('✅ Loading preprocessed data...')
items = pl.read_parquet(items_path)
events = pl.read_parquet(events_path)
tracks_catalog_clean = pl.read_parquet(tracks_catalog_path)

print(f'\n✅ Data loaded successfully!')
print(f'   Items: {items.shape}')
print(f'   Events: {events.shape}')
print(f'   Tracks catalog: {tracks_catalog_clean.shape}')

✅ Loading preprocessed data...

✅ Data loaded successfully!
   Items: (5573129, 9)
   Events: (161854298, 4)
   Tracks catalog: (804763, 6)


## EDA

In [4]:
# ---------- Check data summary ---------- #

def data_summary(df: pl.DataFrame, name: str):
    '''
    Display a quick overview of a Polars DataFrame.
    '''

    print(f'\n===== {name.upper()} =====')  
  
    # Sample rows
    print('\nSample rows:')
    display(df.head())

    # Shape
    rows, cols = df.shape
    print(f'\nShape: {rows:,} rows x {cols} columns')
    
    # Data info
    print('\nSummary for numeric columns:')
    display(df.describe())
   
    # Column info
    print('\nColumn names and types:')
    for col in df.columns:
        print(f'  {col}: {df[col].dtype}')
    
    # Missing values
    print('\nMissing values:')
    display(df.null_count())

In [5]:
# ---------- Items data summary ---------- #

data_summary(items, 'items')


===== ITEMS =====

Sample rows:


track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
i64,i64,i64,i64,str,i64,str,str,str
71759,28614,6118,73,"""Canto De La Tierra""",71759,"""Joy Singers Of Venice""","""Lo Mejor De Andrea Bocelli Viv…","""Vocal"""
103270,7401,19047000,73,"""Here Comes Santa Claus Right D…",103270,"""Bing Crosby""","""It S Christmas 2021""","""Vocal"""
71814,97790,2510714,23,"""Saint Saens Samson Et Dalila O…",71814,"""Marilyn Horne""","""Essential Saint Saens""","""Classical"""
113666,6394,77852,102,"""The Gun""",113666,"""Lou Reed""","""Steel Box Collection Greatest …","""Allrock"""
74091,764,15895239,102,"""She S Got It""",74091,"""Little Richard""","""The Specialty Story""","""Allrock"""



Shape: 5,573,129 rows x 9 columns

Summary for numeric columns:


statistic,track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
str,f64,f64,f64,f64,str,f64,str,str,str
"""count""",5.573129e6,5.573129e6,5.573129e6,5.573129e6,"""5573129""",5.573129e6,"""5573129""","""5573129""","""5573129"""
"""null_count""",0.0,0.0,0.0,0.0,"""0""",0.0,"""0""","""0""","""0"""
"""mean""",2.5724e7,1.7938e6,6.0391e6,79.076673,null,2.5709e7,null,null,null
"""std""",2.2881e7,2.2242e6,5.4570e6,119.376843,null,2.2878e7,null,null,null
"""min""",26.0,4.0,3.0,1.0,"""""",26.0,"""""","""""","""African"""
"""25%""",4.547891e6,49502.0,2.532077e6,19.0,null,4.546055e6,null,null,null
"""50%""",2.353175e7,665453.0,3.362051e6,49.0,null,2.3531749e7,null,null,null
"""75%""",3.6136674e7,3.343278e6,9.745506e6,74.0,null,3.6088784e7,null,null,null
"""max""",1.01495927e8,1.6102782e7,2.1458968e7,1542.0,"""Ꞌcosmicꞌ M4A""",1.01495927e8,"""龍胆紫 Purplesoul""","""Ꞌcosmicꞌ M4A""","""Work"""



Column names and types:
  track_id: Int64
  artist_id: Int64
  album_id: Int64
  genre_id: Int64
  track_clean: String
  track_group_id: Int64
  artist_clean: String
  album_clean: String
  genre_clean: String

Missing values:


track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [6]:
# ---------- Events data summary ---------- #

data_summary(events, 'events')


===== EVENTS =====

Sample rows:


user_id,track_id,listen_count,last_listen
i32,i32,u32,date
2063,22207888,1,2022-06-11
2063,38572747,1,2022-08-13
2064,6995072,1,2022-06-11
2067,3963600,1,2022-04-11
2067,39393063,1,2022-05-24



Shape: 161,854,298 rows x 4 columns

Summary for numeric columns:


statistic,user_id,track_id,listen_count,last_listen
str,f64,f64,f64,str
"""count""",1.61854298e8,1.61854298e8,1.61854298e8,"""161854298"""
"""null_count""",0.0,0.0,0.0,"""0"""
"""mean""",687122.278314,3.7341e7,1.0,"""2022-08-29 12:04:15.569057"""
"""std""",396880.074392,2.7486e7,0.0,null
"""min""",0.0,26.0,1.0,"""2022-01-01"""
"""25%""",343004.0,1.269124e7,1.0,"""2022-07-01"""
"""50%""",687452.0,3.681126e7,1.0,"""2022-09-15"""
"""75%""",1.030412e6,5.8915023e7,1.0,"""2022-11-10"""
"""max""",1.374582e6,1.01495927e8,1.0,"""2022-12-31"""



Column names and types:
  user_id: Int32
  track_id: Int32
  listen_count: UInt32
  last_listen: Date

Missing values:


user_id,track_id,listen_count,last_listen
u32,u32,u32,u32
0,0,0,0


In [7]:
# tracks_catalog_clean already loaded in Cell 5
display(tracks_catalog_clean.head())

track_id,track_clean,track_group_id,artist_id,album_id,genre_id
i64,str,i64,i64,i64,i64
52535988,"""Greaze Mode""",52535988,3353403,7399309,75
44091437,"""Imposible""",44091437,132491,15771801,11
53189466,"""Лето""",53189466,4332353,7604901,51
51966600,"""Summer Splash""",51966600,189166,7283578,68
52708469,"""Етд Кгам""",52708469,4837361,7439209,11


In [8]:
#---------- Top tracks by popularity ---------- #

top_tracks_by_listen_number = (
    events
        .group_by('track_id')
        .agg(pl.sum('listen_count').alias('total_listen_count'))
        .join(tracks_catalog_clean.select(['track_id', 'track_clean']), on='track_id', how='left')
        .sort('total_listen_count', descending=True)
        .head(10)
)

display(top_tracks_by_listen_number)

track_id,total_listen_count,track_clean
i32,u32,str
53404,111062,"""Smells Like Teen Spirit"""
33311009,105666,"""Believer"""
35505245,98962,"""I Got Love"""
24692821,85602,"""Way Down We Go"""
65851540,85173,"""Юность"""
795836,84976,"""Shape Of My Heart"""
32947997,84973,"""Shape Of You"""
45499814,83648,"""Life"""
51241318,83411,"""In The End"""


In [9]:
# Memory-efficient approach: aggregate first (small result), then join for names
print("=== Top Track Versions ===")

# Step 1: Get top 10 track IDs by listen count (small DataFrame)
top_track_ids = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('total_listens'))
    .sort('total_listens', descending=False)
    .tail(10)  # Use tail for descending (more efficient)
    .collect()
)

# Step 2: Join only these 10 tracks with catalog (small join)
top_track_versions = (
    top_track_ids
    .join(tracks_catalog_clean.select(['track_id', 'track_clean', 'track_group_id']), 
          on='track_id', how='left')
    .sort('total_listens', descending=True)
)
display(top_track_versions)

# 2. Most popular songs (versions aggregated) - memory efficient
print("\n=== Top Songs (All Versions Combined) ===")

# Step 1: Aggregate by track_id first (reduces size before join)
events_agg = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('track_listens'))
    .collect()
)

# Step 2: Join with catalog (smaller join)
events_with_groups = (
    events_agg
    .join(tracks_catalog_clean.select(['track_id', 'track_clean', 'track_group_id']), 
          on='track_id', how='left')
)

# Step 3: Group by track_group_id
top_songs = (
    events_with_groups
    .group_by(['track_group_id', 'track_clean'])
    .agg([
        pl.sum('track_listens').alias('total_listens'),
        pl.len().alias('num_versions'),
    ])
    .sort('total_listens', descending=True)
    .head(10)
)
display(top_songs)

# 3. Analysis: How many top tracks are just different versions?
print("\n=== Diversity Analysis ===")

# Get top 100 tracks
top_100_track_ids = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('total_listens'))
    .sort('total_listens', descending=False)
    .tail(100)
    .collect()
)

top_100_with_groups = (
    top_100_track_ids
    .join(tracks_catalog_clean.select(['track_id', 'track_group_id']), 
          on='track_id', how='left')
)

unique_groups_in_top_100 = top_100_with_groups['track_group_id'].n_unique()
print(f"Top 100 tracks represent {unique_groups_in_top_100} unique songs")
print(f"Duplication rate: {(100 - unique_groups_in_top_100) / 100:.1%}")

=== Top Track Versions ===


track_id,total_listens,track_clean,track_group_id
i32,u32,str,i64
53404,111062,"""Smells Like Teen Spirit""",53404
33311009,105666,"""Believer""",33311009
35505245,98962,"""I Got Love""",35505245
24692821,85602,"""Way Down We Go""",24692821
65851540,85173,"""Юность""",65851540
795836,84976,"""Shape Of My Heart""",795836
32947997,84973,"""Shape Of You""",32947997
45499814,83648,"""Life""",45499814
51241318,83411,"""In The End""",51241318



=== Top Songs (All Versions Combined) ===


track_group_id,track_clean,total_listens,num_versions
i64,str,u32,u32
53404,"""Smells Like Teen Spirit""",111062,1
33311009,"""Believer""",105666,1
35505245,"""I Got Love""",98962,1
24692821,"""Way Down We Go""",85602,1
65851540,"""Юность""",85173,1
795836,"""Shape Of My Heart""",84976,1
32947997,"""Shape Of You""",84973,1
45499814,"""Life""",83648,1
51241318,"""In The End""",83411,1



=== Diversity Analysis ===
Top 100 tracks represent 100 unique songs
Duplication rate: 0.0%


Наиболее популярные жанры

In [10]:
# Top 5 genres by listening number - memory efficient
# Step 1: Aggregate events by track_id (reduces size)
events_by_track = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('track_listen_count'))
    .collect()
)

# Step 2: Get unique track-genre mapping (small subset of items)
track_genres = items.select(['track_id', 'genre_clean']).unique(['track_id', 'genre_clean'])

# Step 3: Join and aggregate
genres_by_listen_count = (
    events_by_track
    .join(track_genres, on='track_id', how='left')
    .group_by('genre_clean')
    .agg(pl.sum('track_listen_count').alias('total_listen_count'))
    .sort('total_listen_count', descending=True)
)

top_5_genres = genres_by_listen_count.head(5)
display(top_5_genres)

genre_clean,total_listen_count
str,u32
"""Pop""",47600763
"""Rap""",30960775
"""Ruspop""",21954977
"""Allrock""",21181228
"""Rusrap""",20437496


Треки, которые никто не прослушал

In [11]:
# Tracks that haven't been listened to by anybody - memory efficient
# Step 1: Get set of listened track IDs (small set)
listened_track_ids = set(events['track_id'].unique())

# Step 2: Get unique tracks from items
unique_tracks = items.select(['track_id', 'track_clean', 'artist_clean', 'album_clean', 'genre_clean']).unique('track_id')

# Step 3: Filter out listened tracks (using Python set for efficiency)
unlistened_tracks = unique_tracks.filter(~pl.col('track_id').is_in(list(listened_track_ids)))

print(f'Number of unlistened tracks: {unlistened_tracks.height:,}')
display(unlistened_tracks.head(10))

Number of unlistened tracks: 50


track_id,track_clean,artist_clean,album_clean,genre_clean
i64,str,str,str,str
46786298,"""Heresy Blind""","""Nine Inch Nails""","""Closer To God""","""Industrial"""
56166858,"""Панк Туристы""","""Скамеика Запасных""","""Новые Грехи Старые Традиции""","""Punk"""
77751634,"""Gypsy Drum Solo""","""Dio""","""Holy Diver""","""Classicmetal"""
73459486,"""Lord Of The Flies""","""Elton John""","""Jewel Box""","""Pop"""
61606834,"""В Любои Момент Могу Бросить""","""Вбензинеестьоблака""","""Все Что Вы Хотели Но Боялись Е…","""Rap"""
70050701,"""Symphonic Suite Kiki S Deliver…","""New Japan Philharmonic World D…","""Symphonic Suite Kiki S Deliver…","""Modern"""
97264237,"""Рождество""","""Арт Группа Largo""","""Рождество""","""Spiritual"""
88466656,"""Last Horizon""","""Brian May""","""Back To The Light""","""Allrock"""
57627360,"""Statues""","""Death From Above 1979""","""Outrage Is Now""","""Alternative"""


In [16]:
import pickle, polars as pl
with open('../models/label_encoders.pkl','rb') as f:
    enc = pickle.load(f)
user_enc = enc['user_encoder']
track_enc = enc['track_encoder']

# 1. Type & a few items
print(type(user_enc), list(user_enc.items())[:5])
print(type(track_enc), list(track_enc.items())[:5])

# 2. How many mappings? are keys user_ids (e.g. 2063) or indices (0,1,2..)?
print('user keys sample:', list(user_enc.keys())[:10])
print('track keys sample:', list(track_enc.keys())[:10])

# 3. Compare with raw events
events = pl.read_parquet('../data/preprocessed/events.parquet')
print('unique users in events:', events['user_id'].n_unique())
print('unique tracks in events:', events['track_id'].n_unique())

# 4. Try a replace on a sample to see how many become null
sample = events.head(10000)
mapped = sample.with_columns([
    pl.col('user_id').replace(user_enc).alias('user_idx'),
    pl.col('track_id').replace(track_enc).alias('track_idx'),
])
print('null user_idx:', mapped['user_idx'].null_count(), 'null track_idx:', mapped['track_idx'].null_count())


<class 'dict'> [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
<class 'dict'> [(26, 0), (38, 1), (135, 2), (136, 3), (138, 4)]
user keys sample: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
track keys sample: [26, 38, 135, 136, 138, 139, 140, 141, 143, 144]
unique users in events: 1371770
unique tracks in events: 804713
null user_idx: 0 null track_idx: 0
